## 5.1 环境配置

首先，我们导入必须的环境，数据集和model使用和前几章相同的minist数据集和LeNet网络。


In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.quantization
import random
import numpy as np
import torch.nn as nn
import copy
from torch.optim import *
from torch.optim.lr_scheduler import *
from torch.utils.data import DataLoader
from torchprofile import profile_macs
from torchvision.datasets import *
from torchvision.transforms import *
import torch.nn.functional as F
from torchvision import datasets
from utils import LeNet,train,evaluate,get_model_size,get_model_flops,MiB

random.seed(0)
np.random.seed(0)
torch.manual_seed(0)

## 5.2 构建模型和数据集

In [2]:
class LeNet(nn.Module):
    def __init__(self, num_classes=10):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5)
        self.relu1 = nn.ReLU()
    
        self.maxpool1 = nn.MaxPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5)
        self.relu2 = nn.ReLU()
        
        self.maxpool2 = nn.MaxPool2d(kernel_size=2, stride=2)
        
        self.fc1 = nn.Linear(in_features=16 * 4 * 4, out_features=120)
        self.relu3 = nn.ReLU()
        
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.relu4 = nn.ReLU()
        
        self.fc3 = nn.Linear(in_features=84, out_features=num_classes)
        
        self.quant = torch.quantization.QuantStub()
        self.dequant = torch.quantization.DeQuantStub()
       
    def forward(self, x):
        x = self.quant(x)
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))

        x = x.contiguous().view(x.shape[0], -1)
        x = self.relu3(self.fc1(x))
        x = self.relu4(self.fc2(x))
        x = self.fc3(x)
        x = self.dequant(x)
        return x

In [3]:
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LeNet()#.to(device=device)
print(model)
# 加载模型的状态字典
data_dir = "../../01prune/notebook/0.minist_classify"
checkpoint = torch.load(data_dir+'/model.pt')
# checkpoint = torch.load('./model.pt')

# 加载状态字典到模型
model.load_state_dict(checkpoint)
fp32_model = copy.deepcopy(model)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (relu3): ReLU()
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (relu4): ReLU()
  (fc3): Linear(in_features=84, out_features=10, bias=True)
  (quant): QuantStub()
  (dequant): DeQuantStub()
)


/tmp/ipykernel_3546/3723449019.py:6: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(data_dir+'/model.pt')


In [4]:
# 设置归一化
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))])

# 获取数据集
calib_dataset = datasets.MNIST(root=data_dir+'/data/mnist', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(root=data_dir+'/data/mnist', train=False, download=True, transform=transform)  # train=True训练集，=False测试集
# 设置DataLoader
batch_size = 64
calib_loader = DataLoader(calib_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 5.3 验证 FP32 模型的精度以及模型大小

In [5]:
fp32_model_accuracy = evaluate(fp32_model, test_loader)
fp32_model_size = get_model_size(fp32_model)
print(f"fp32 model has accuracy={fp32_model_accuracy:.5f}%")
print(f"fp32 model has size={fp32_model_size/MiB:.2f} MiB")

eval:   0%|          | 0/157 [00:00<?, ?it/s]

fp32 model has accuracy=97.99000%
fp32 model has size=0.17 MiB


## 5.4 将模型转化为量化模型

In [6]:
backend = torch.backends.quantized.supported_engines #运行时可以使用这句命令检查自己支持的后端
print(backend)

['qnnpack', 'none', 'onednn', 'x86', 'fbgemm']


## 5.5 使用测试集进行校准

In [7]:
# 配置量化
quant_model = copy.deepcopy(fp32_model)
quant_model.qconfig = torch.quantization.get_default_qconfig('fbgemm')
torch.quantization.prepare(quant_model, inplace=True)

# 使用校准数据跑一遍前向传播
def calibrate(model, loader, num_batches=batch_size):
    model.eval()
    with torch.no_grad():
        for i, (data, target) in enumerate(loader):
            if i >= num_batches:
                break
            model(data)

calibrate(quant_model, calib_loader)

/home/tony/miniconda3/envs/pt11.8/lib/python3.10/site-packages/torch/ao/quantization/observer.py:229: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


## 5.6 验证量化模型的精度

In [8]:
quant_model_accuracy = evaluate(quant_model, test_loader)
print(f"quant model has accuracy={quant_model_accuracy:.5f}%")


eval:   0%|          | 0/157 [00:00<?, ?it/s]

quant model has accuracy=97.99000%


## 5.7 将量化模型转化为量化部署模型并验证其精度

这里重新定义一个验证函数。因为模型在量化后，输入输出也需要进行量化。具体添加的内容如下：
    outputs = dequant(outputs)
    inputs = quant(inputs)

In [9]:
from tqdm.auto import tqdm
quant = torch.quantization.QuantStub() 
dequant = torch.quantization.DeQuantStub()

def evaluate_quant(
  model: nn.Module,
  dataloader: DataLoader,
  extra_preprocess = None
) -> float:
  model.eval()

  num_samples = 0
  num_correct = 0
  
  for inputs, targets in tqdm(dataloader, desc="eval", leave=False):
    # Move the data from CPU to GPU
    # inputs = inputs.to('mps')
    if extra_preprocess is not None:
        for preprocess in extra_preprocess:
            inputs = quant(inputs)
            inputs = preprocess(inputs)

    # targets = targets.to('mps')

    # Inference
    outputs = model(inputs)
    # print(outputs)
    outputs = dequant(outputs)
    # Convert logits to class indices
    # print(outputs)
    outputs = outputs.to("cpu")
    # outputs = outputs.argmax(dim=1)
    outputs=torch.max(outputs,1)[1]

    # Update metrics
    num_samples += targets.size(0)
    num_correct += (outputs == targets).sum()

  return (num_correct / num_samples * 100).item()

将量化训练后的模型中的Fake节点去掉，这样可以得到一个真正的int8模型。下边模型中的节点算子都变成了Quantized算子。

In [10]:
torch.quantization.convert(quant_model, inplace=True)
print(quant_model)


LeNet(
  (conv1): QuantizedConv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), scale=0.15729273855686188, zero_point=35)
  (relu1): ReLU()
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): QuantizedConv2d(6, 16, kernel_size=(5, 5), stride=(1, 1), scale=0.34748244285583496, zero_point=57)
  (relu2): ReLU()
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (fc1): QuantizedLinear(in_features=256, out_features=120, scale=0.2727542221546173, zero_point=51, qscheme=torch.per_channel_affine)
  (relu3): ReLU()
  (fc2): QuantizedLinear(in_features=120, out_features=84, scale=0.22233228385448456, zero_point=52, qscheme=torch.per_channel_affine)
  (relu4): ReLU()
  (fc3): QuantizedLinear(in_features=84, out_features=10, scale=0.3006247580051422, zero_point=63, qscheme=torch.per_channel_affine)
  (quant): Quantize(scale=tensor([0.0255]), zero_point=tensor([17]), dtype=torch.quint8)
  (dequant): DeQuantize()
)


## 验证全int8模型的精度

可以看到全int8模型的精度和量化训练时的最后一个epoch的精度几乎一致。

In [11]:
final_quant_model_accuracy = evaluate_quant(quant_model, test_loader)
print(f"final_quant model has accuracy={final_quant_model_accuracy:.5f}%")

eval:   0%|          | 0/157 [00:00<?, ?it/s]

final_quant model has accuracy=97.88000%


In [12]:
def save_onnx(model, model_name):
    dummy_input = torch.randn(1, 1, 28, 28)  
    torch.onnx.export(model,
                      dummy_input,
                      f"{model_name}.onnx", 
                      input_names=["input"],
                      output_names=["output"],
                      dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}})

In [13]:
# 保存量化后的模型
save_onnx(quant_model, "quant_model")

In [14]:
# 保存量化前的onnx模型
save_onnx(fp32_model, "fp32_model")